# Data Collection
- Get list of 20 most visited cities.
- Create a dictionary with coordinates of 20 most visited cities.

In [1]:
import tqdm

### Get a list of the 20 most visited cities:

In [2]:
import requests
from scrapy import Selector
import json
import custom_functions as cf

In [3]:
cities_url = "https://travelness.com/most-visited-cities-in-the-world" # URL of the page with the list of cities

response = requests.get(cities_url)
sel = Selector(response)

cities = sel.xpath("//table//tr/td[2]/text()").getall()

### Use xxx API to convert the city names into coordinates

In [4]:
from geopy.geocoders import Nominatim

In [25]:
def geocode_city(city):
    """
    Geocode a single city using OpenStreetMap API.
    """
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city)
    return {"city": city, "latitude": location.latitude, "longitude": location.longitude}

def geocode_cities(city_list):
    """
    Geocode a list of cities using OpenStreetMap API.
    """
    geocoded_cities = [geocode_city(city) for city in city_list if geocode_city(city)]
    return geocoded_cities

# Geocode the list of cities
geocoded_cities = geocode_cities(cities)

In [6]:
# export geocoded cities to a json file
with open("../data/city_coordinates.json", "w") as f:
    json.dump(geocoded_cities, f)

### Use the open-meteo API to get weather data

In [8]:
# import relevant modules
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [9]:
# Define our variables of interest as a list
daily_variables_of_interest = [
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "daylight_duration",
    "sunshine_duration",
    "precipitation_sum",
    "rain_sum",
    "precipitation_hours",
]

# Create the API params dictionary
params = {
    "latitude": [city["latitude"] for city in geocoded_cities],
    "longitude": [city["longitude"] for city in geocoded_cities],
    "start_date": "1940-01-01",
    "end_date": "2023-12-31",
    "daily": daily_variables_of_interest,
}

In [10]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
responses = openmeteo.weather_api(url, params=params)

In [85]:
i = 14
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[i]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
daily = response.Daily()
temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daylight_duration = daily.Variables(3).ValuesAsNumpy()
sunshine_duration = daily.Variables(4).ValuesAsNumpy()
precipitation_sum = daily.Variables(5).ValuesAsNumpy()
rain_sum = daily.Variables(6).ValuesAsNumpy()
precipitation_hours = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}

daily_data["city"] = geocoded_cities[i]['city']
daily_data["temperature_2m_max"] = temperature_2m_max
daily_data["temperature_2m_min"] = temperature_2m_min
daily_data["temperature_2m_mean"] = temperature_2m_mean
daily_data["daylight_duration"] = daylight_duration
daily_data["sunshine_duration"] = sunshine_duration
daily_data["precipitation_sum"] = precipitation_sum
daily_data["rain_sum"] = rain_sum
daily_data["precipitation_hours"] = precipitation_hours

daily_dataframe = pd.DataFrame(data = daily_data)
display(daily_dataframe)

Coordinates 12.970123291015625°N 100.96363067626953°E
Elevation 10.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,city,temperature_2m_max,temperature_2m_min,temperature_2m_mean,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,precipitation_hours
0,1940-01-01 00:00:00+00:00,Pattaya,26.555500,16.605499,21.205500,40955.824219,NaN,NaN,NaN,0.0
1,1940-01-02 00:00:00+00:00,Pattaya,26.455500,15.905500,20.974251,40966.664062,38279.835938,0.0,0.0,0.0
2,1940-01-03 00:00:00+00:00,Pattaya,26.205500,15.655500,20.393000,40978.398438,38293.085938,0.0,0.0,0.0
3,1940-01-04 00:00:00+00:00,Pattaya,27.505501,15.805500,20.903418,40991.015625,38307.332031,0.0,0.0,0.0
4,1940-01-05 00:00:00+00:00,Pattaya,28.505501,16.005499,21.961752,41004.503906,38322.535156,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
30676,2023-12-27 00:00:00+00:00,Pattaya,32.129501,21.929501,27.146166,40913.328125,38121.406250,0.0,0.0,0.0
30677,2023-12-28 00:00:00+00:00,Pattaya,32.179501,23.579498,27.925333,40919.398438,37599.816406,0.0,0.0,0.0
30678,2023-12-29 00:00:00+00:00,Pattaya,33.479500,24.679501,28.306585,40926.496094,38187.410156,0.2,0.2,2.0
30679,2023-12-30 00:00:00+00:00,Pattaya,32.579498,24.229500,28.158663,40934.589844,38221.445312,0.1,0.1,1.0


In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [39.55, 19.0728, 22.5455, 36.9081, 41.0138, 3.1412, 25.0772, 1.2897, 51.5085, 22.2783, 48.8534, 28.6519, 35.6895, 25.0478, 12.9333, 41.8919, 40.7143, 13.754],
    "longitude": [2.7333, 72.8826, 114.0683, 30.6956, 28.9497, 101.6865, 55.3093, 103.8501, -0.1257, 114.1747, 2.3488, 77.2315, 139.6917, 121.5319, 100.8833, 12.5113, -74.006, 100.5014],
    "start_date": "1940-01-01",
    "end_date": "2024-03-01",
    "daily": ["precipitation_sum", "rain_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)


# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
daily_rain_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe = pd.DataFrame(data = daily_data)
display(daily_dataframe)


Coordinates 39.54305648803711°N 2.742382287979126°E
Elevation 2.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,precipitation_sum,rain_sum,precipitation_hours
0,1940-01-01 00:00:00+00:00,NaN,NaN,0.0
1,1940-01-02 00:00:00+00:00,1.300000,1.300000,6.0
2,1940-01-03 00:00:00+00:00,6.400000,6.400000,13.0
3,1940-01-04 00:00:00+00:00,2.000000,2.000000,8.0
4,1940-01-05 00:00:00+00:00,0.000000,0.000000,0.0
...,...,...,...,...
30737,2024-02-26 00:00:00+00:00,8.400000,8.400000,8.0
30738,2024-02-27 00:00:00+00:00,0.400000,0.400000,4.0
30739,2024-02-28 00:00:00+00:00,6.099999,6.099999,14.0
30740,2024-02-29 00:00:00+00:00,0.000000,0.000000,0.0


,precipitation_sum,rain_sum,precipitation_hours
count,30741.000000,30741.000000,30742.000000
mean,1.088254,1.085811,2.256359
std,3.344872,3.334524,4.256175
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.500000,0.500000,3.000000
max,90.099991,90.099991,24.000000


In [ ]:
# doing it just for London 


import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 51.5085,
    "longitude": -0.1257,
    "start_date": "1940-01-01",
    "end_date": "2024-03-01",
    "daily": ["precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe = pd.DataFrame(data = daily_data)
display(daily_dataframe)


OpenMeteoRequestsError: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}